In [8]:
import os
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import ee
import rs
ee.Initialize()

In [9]:
# This function adds a time band to the image.
def createTimeBand(image):
  # Scale milliseconds by a large constant.
  image.set('monsoon_year', image.get('system:time_start').get('year'))
  image.set('year_diff', ee.Number(image.get('system:time_start'))
            .subtract(ee.Date('2015-07-29').millis()).divide(3.1536e10))
  return image.addBands(image.metadata('year_diff'))

# This function adds a constant band to the image.
def createConstantBand(image):
  return ee.Image(1).addBands(image)




In [10]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
ee_geometry = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))

tm5 = rs.prep_tm5_ic(ee_geometry).filterDate('2014-07-01','2015-07-01')
etm7 = rs.prep_etm7_ic(ee_geometry).filterDate('2014-07-01','2015-07-01')
oli8 = rs.prep_oli8_ic(ee_geometry).filterDate('2014-07-01','2015-07-01')


# etm7.map(createTimeBand)

landsat = tm5.merge(etm7).merge(oli8) \
    .filter(ee.Filter.lt('CLOUD_COVER', 50))

landsat_clouds = rs.prep_landsat_clouds(landsat)

In [5]:
from math import pi
3*pi
6*pi

18.84955592153876

In [11]:
def addRegressionBands(img):
    # (1e3 * 60 * 60 * 24 * 365) #/ 3.1536e10 # milliseconds in year
    start_month = 6  # e.g., 2014-07-01
    img_date = ee.Date(img.get('system:time_start'))
    year = img_date.get('year')
    prev_year = img_date.get('month').subtract(start_month).lt(0)
    monsoon_year = ee.Number(year.subtract(prev_year))
    year_start_date = ee.Date.fromYMD(monsoon_year, start_month, 1)
    year_frac = img_date.millis().subtract(
        year_start_date.millis()).divide(3.1536e10)
    # img = img.set('year_frac', year_frac)
    img = img.set('year_frac', year_frac)
    year_frac = img.metadata('year_frac')
    time3pi = year_frac.multiply(9.42477796076938)
    time6pi = year_frac.multiply(18.84955592153876)
    img = img \
        .addBands(year_frac) \
        .addBands(ee.Image(1).rename('constant')) \
        .addBands(time3pi.cos().rename('cos3pi')) \
        .addBands(time3pi.sin().rename('sin3pi')) \
        .addBands(time6pi.cos().rename('cos6pi')) \
        .addBands(time6pi.sin().rename('sin6pi'))
    # img = img.addBands(
    #     img.select('year_frac')
    return img


In [12]:
landsat = landsat.map(addRegressionBands)
landsat.first().getInfo()
# date_2014_07_28 = ee.Date('2014-07-28')
# img = etm7.filterDate('2014-07-01','2014-08-01').first() # 07-29

# start_month = 7 # e.g., 2014-07-01
# img_date = ee.Date(img.get('system:time_start'))
# year = img_date.get('year')
# prev_year = img_date.get('month').subtract(start_month).lt(0)
# monsoon_year = ee.Number(year.subtract(prev_year))
# year_start_date = ee.Date.fromYMD(monsoon_year, start_month, 1)
# img = img.set('year_frac', img_date.millis().subtract(year_start_date.millis()).divide(3.1536e10))
# img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'swir2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7991, 6941],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1544415]},
  {'id': 'swir1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7991, 6941],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1544415]},
  {'id': 'nir',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7991, 6941],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1544415]},
  {'id': 'red',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7991, 6941],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1544415]},
  {'id': 'green',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [20]:
44/365

0.12054794520547946

In [11]:

# Load the input image collection: projected climate data.
collection = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS') 
  .filterDate(ee.Date('2006-01-01'), ee.Date('2099-01-01')) 
  .filter(ee.Filter.eq('scenario', 'rcp85')) 
  .map(createTimeBand) # Map the functions over the collection, to get constant and time bands.
  .map(createConstantBand)
  .select(['constant', 'system:time_start', 'pr_mean', 'tasmax_mean'])) # Select the predictors and the responses.

# Compute ordinary least squares regression coefficients.
linearRegression = collection.reduce(
  ee.Reducer.linearRegression({
    'numX': ee.Number(2),
    'numY': ee.Number(2)
}))

EEException: Invalid argument specified for ee.Number(): {'numX': <ee.ee_number.Number object at 0x120b90fd0>, 'numY': <ee.ee_number.Number object at 0x1209234f0>}

In [1]:
collection = ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')#
collection

NameError: name 'ee' is not defined